# Über dieses Notebook

Dieses Notebook begleitet meine Module _Computational Biology_ und _Biodata Processing_ an der Hochschule Mittweida. Wenn Sie _Jupyter_ installiert haben, können Sie diese Datei herunterladen und nutzen. Ansonsten kopieren Sie die Befehle einfach in Ihren Terminal. In jedem Fall brauchen Sie ein Linux-System – echt oder emuliert.

Mit den Skripten in diesem Notebook werden wir die __Sequenzvariationen von SARS-CoV-2 Viren__ analysieren. Die Analyse ist dahingehend vereinfacht, dass wir nur Genome einer bestimmten Länge untersuchen, die beim NCBI hinterlegt sind.

Have a lot of fun ...

# Technische Information

- `!` ruft eine Bash-Shell als einen neuen Prozess auf. Nach dessen Ausführung wird die Shell direkt wieder geschlossen. Daher funktioniert `! cd XXX` nicht, da zwar in das Verzeichnis _XXX_ gewechselt, die Shell aber direkt wieder geschlossen wird.

- `%` beeinflusst den mit dem Notebook verbundenen Prozess, also das Notebook selbst. `%cd XXX` ändert daher das aktuelle Verzeichnis des Notebook-Prozesses. ACHTUNG: keine Leerzeichen nach `%`

- Meisten wollen Sie `!`anwenden! Bei `cd` aber `%`.

- So lange nach der Ausführung eines Befehls vor der Zeile noch ein `*` (__In[*]__) und keine Zeilennummer steht, ist der Befehl noch nicht abgeschlossen.

- <span style="color:red">Achtung: Ihre Ergebnisse können von meinen abweichen, da die Daten ständig aktualisiert werden.</span>

---

# Download des Referenzgenoms

Das Referenzgenom für SARS-CoV-2 wurde im Dezember 2019 in Wuhan isoliert und sequenziert. 

- Gehen Sie zur NCBI Webseite: https://www.ncbi.nlm.nih.gov/sars-cov-2/.

- Klicken Sie im Abschnitt _Genome Reference Sequence (NC_045512)_ auf __View Record__ weiter unten.

- Klicken Sie in dem neuen Fenster oben auf __Send to__ und wählen Sie dort 
  - _Complete Record_
  - Destination _file_
  - Format _fasta_
  
- Speichern Sie die Datei als _NC_045512.2.fasta_ in Ihr Arbeitsverzeichnis _SARS-CoV-2_.
  
- Klicken Sie nochmal auf __Send to__ und wählen Sie jetzt als Format
  - _GFF3_

- Speichern Sie die Datei als _NC_045512.2.gff3_ in Ihr Arbeitsverzeichnis _SARS-CoV-2_.

# Download der Virusgenome

- Gehen Sie zur NCBI Webseite: https://www.ncbi.nlm.nih.gov/sars-cov-2/.

- Klicken Sie im Abschnitt _Explore the Data_ auf __Explore in NCBI Virus__ weiter unten.

- NC_045512 ist das Referenzgenom mit 29.903 Nukleotiden.

- Filtern Sie die Tabelle nach _completeness_ und _length = 29903_ Nukleotide. Mit Stand vom 01/02/21 werden 1.152 Sequenzen ausgewählt.

- Klicken Sie _Download_, select _Nucleotide FASTA_, select _All Records_, select _Build custom_ und wählen Sie hier nur den Eintrag _Accession_

- Verschieben Sie heruntergeladene Datei in Ihr Arbeitsverzeichnis _SARS-CoV-2_ und benennen Sie die Datei in _cov2-len-29903.fasta_ um

| ![Reference Sequence](Images/refseq.png)
|:---
| Download von NCBI Genbank

In [131]:
! pwd

/Users/rw/Playgound/Jupyter/SARS-CoV-2/BWA


- zählen Sie die Anzahl der enthaltenden Sequenzen:

In [132]:
! grep -c ">" cov2-len-29903.fasta

1151


---

# Suche nach Mutationen mit AWK

Mit Hilfe von AWK können wir die Genome direkt bearbeiten. Dazu brauchen Sie die zwei Scripte _compare-cov2.awk_ und _fasta2tbl_ von meiner __GitHub__ Seite: https://github.com/awkologist/SARS-CoV-2. Diese können Sie direkt mit `wget` herunterladen:

In [94]:
! wget 'https://raw.githubusercontent.com/awkologist/SARS-CoV-2/main/fasta2tbl'

--2021-01-30 21:04:08--  https://raw.githubusercontent.com/awkologist/SARS-CoV-2/main/fasta2tbl
Resolving raw.githubusercontent.com... 151.101.12.133
Connecting to raw.githubusercontent.com|151.101.12.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 299 [text/plain]
Saving to: 'fasta2tbl'

fasta2tbl           100%[===================>]     299  --.-KB/s    in 0s      

2021-01-30 21:04:08 (14.3 MB/s) - 'fasta2tbl' saved [299/299]



In [95]:
! wget 'https://raw.githubusercontent.com/awkologist/SARS-CoV-2/main/compare-cov2.awk'

--2021-02-01 08:39:00--  https://raw.githubusercontent.com/awkologist/SARS-CoV-2/main/compare-cov2.awk
Resolving raw.githubusercontent.com... 151.101.112.133
Connecting to raw.githubusercontent.com|151.101.112.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2727 (2.7K) [text/plain]
Saving to: 'compare-cov2.awk'

compare-cov2.awk    100%[===================>]   2.66K  --.-KB/s    in 0.001s  

2021-02-01 08:39:01 (2.24 MB/s) - 'compare-cov2.awk' saved [2727/2727]



- Da AWK Dateien zeilenweise liest und jede Zeile in Felder zerlegt, ist das FASTA-Format nicht gut für die Datenanalyse mit AWK geeignet. Mit den Script _fasta2tbl_ wandeln wir daher das FASTA- in das TAB-Format, also mit Tabulatoren als Separatoren.

In [110]:
! ./fasta2tbl cov2-len-29903.fasta > cov2-len-29903.tab

- Upps. Wir müssen das Script erst ausführbar machen:

In [98]:
! ls -l fasta2tbl

-rw-r--r--  1 rw  staff  299 Jan 30 21:04 fasta2tbl


In [100]:
! chmod u+x fasta2tbl

In [101]:
! ls -l fasta2tbl

-rwxr--r--  1 rw  staff  299 Jan 30 21:04 fasta2tbl


In [105]:
! ./fasta2tbl cov2-len-29903.fasta > cov2-len-29903.tab

- Jetzt müssen wir noch die Metadaten ins TAB-Format bringen. Diese verwenden wir später im Script _compare-cov2.awk_. Die drei Felder ID, Ort und Datum sind in _cov2-len-29903.csv_ durch Kommata getrennt.

- Es gibt dort aber problematische Zeilen wie _MW276480,"USA: CA, Marin County",2020-08-26_ und _MW011767,"China: Hunan,Shaoyang",2020-01-29_ -> hier ist jeweils ein Komma in der Ortsbezeichnung, mal mit mal ohne nachfolgendem Leerzeichen. Und die Anführungszeichen wollen wir auch löschen. 

| ![Virus DB](Images/ncbi-virus.png)
|:---
| Die Metadaten in _Vim_

- Wir können das elegant mit _Vim_ lösen oder folgendem Einzeiler, der auch gleich den Header löscht:

In [133]:
! egrep '(MW276480|MW011767)' cov2-len-29903.csv

MW276480,"USA: CA, Marin County",2020-08-26
MW011767,"China: Hunan,Shaoyang",2020-01-29


In [134]:
! sed -E -e '1d' -e 's/"(.*), ?(.*)"/\1_\2/' -e 's/,/\t/g' cov2-len-29903.csv > cov2-len-29903.metadata.tab

In [135]:
! egrep '(MW276480|MW011767)' cov2-len-29903.metadata.tab

MW276480	USA: CA_Marin County	2020-08-26
MW011767	China: Hunan_Shaoyang	2020-01-29


- suchen wir zunächst Sequenzen aus Deutschland

In [137]:
! egrep 'Germany' cov2-len-29903.metadata.tab

MT358638	Germany	2020-02
MT358639	Germany	2020-02
MT358640	Germany	2020-02
MT358641	Germany	2020-02
MT358642	Germany	2020-02
MT358643	Germany	2020-02


- Spielen wir nun mit dem AWK-Skript um zunächst zwei Virengenome zu vergleichen:

In [138]:
! awk -f compare-cov2.awk

  Run as:
    awk -f compare-cov2.awk -v seq=seqs.tab -v anno=annos.tab -v id1=ID1 -v id2=ID2
  where seqs.tab is the sequence file in tab format and
        annos.tab is the annotation file in tab format.
  If -v id1=... is omitted, the reference genome ID NC_045512.2 will be used.


In [139]:
! awk -f compare-cov2.awk -v seq=cov2-len-29903.tab -v anno=cov2-len-29903.metadata.tab -v id2=MT358638

# NC_045512.2 > MT358638.1 | 2019-12 > 2020-02 | China > Germany
MT358638.1 3518: G>T 
MT358638.1 12704: G>T 
MT358638.1 12797: G>A 
MT358638.1 17423: A>G 
MT358638.1 27272: T>C 
MT358638.1 28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903


- jetzt vergleichen wir alle deutschen Genome gegen Wuhan 1

In [158]:
! for i in MT358638 MT358639 MT358640 MT358641 MT358642 MT358643; do awk -f compare-cov2.awk -v seq=cov2-len-29903.tab -v anno=cov2-len-29903.metadata.tab -v id2=$i; echo "----"; done

# NC_045512.2 > MT358638.1 | 2019-12 > 2020-02 | China > Germany
MT358638.1 3518: G>T 
MT358638.1 12704: G>T 
MT358638.1 12797: G>A 
MT358638.1 17423: A>G 
MT358638.1 27272: T>C 
MT358638.1 28854: C>T 
# of N's: 0 of 29903
# of exchanges: 6 of 29903
----
# NC_045512.2 > MT358639.1 | 2019-12 > 2020-02 | China > Germany
MT358639.1 241: C>T 
MT358639.1 3037: C>T 
MT358639.1 14408: C>T 
MT358639.1 23403: A>G SPIKE NT:1841 AA:614 -> D
MT358639.1 28881: G>A 
MT358639.1 28882: G>A 
MT358639.1 28883: G>C 
# of N's: 0 of 29903
# of exchanges: 7 of 29903
----
# NC_045512.2 > MT358640.1 | 2019-12 > 2020-02 | China > Germany
MT358640.1 241: C>T 
MT358640.1 3037: C>T 
MT358640.1 6228: A>G 
MT358640.1 14408: C>T 
MT358640.1 16289: C>Y 
MT358640.1 23403: A>G SPIKE NT:1841 AA:614 -> D
MT358640.1 28881: G>A 
MT358640.1 28882: G>A 
MT358640.1 28883: G>C 
MT358640.1 28961: C>T 
MT358640.1 29870: C>M 
# of N's: 0 of 29903
# of exchanges: 11 of 29903
----
# NC_045512.2 > MT358641.1 | 2019-12 > 2020-02 | Ch

- Wir sehen, dass die betroffenen Aminosäuren im Spike-Protein und ggf. in der _receptor binding domain_ (RBDomain) oder im _receptor binding motif_ (RBMotif)

- Mutationen im _receptor binding motif_ haben die größte Wahrscheinlichkeit, zu einer Veränderung der Infektiosität zuführen. Also, analysieren wir nun alle Sequenzen. Dabei erstellt uns der AWK-Befehl in `$()` die Liste. Achtung, die Ausführung dauert ein Minütchen oder zwei.

In [159]:
! for i in $(awk '{print $1}' cov2-len-29903.metadata.tab); do awk -f compare-cov2.awk -v seq=cov2-len-29903.tab -v anno=cov2-len-29903.metadata.tab -v id2=$i; echo "# ----"; done > result.txt

- jetzt extrahieren wir alle _receptor binding motif_ Mutationen

In [160]:
! egrep 'Motif' result.txt

MW493859.1 23010: T>C SPIKE NT:1448 AA:483 -> V RBDomain RBMotif
MW494311.1 23010: T>C SPIKE NT:1448 AA:483 -> V RBDomain RBMotif
MW482861.1 22920: A>T SPIKE NT:1358 AA:453 -> Y RBDomain RBMotif
MW467420.1 23081: C>T SPIKE NT:1519 AA:507 -> P RBDomain RBMotif
MW467456.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW467497.1 22916: C>A SPIKE NT:1354 AA:452 -> L RBDomain RBMotif
MW454554.1 22998: C>T SPIKE NT:1436 AA:479 -> P RBDomain RBMotif
MW454555.1 22998: C>T SPIKE NT:1436 AA:479 -> P RBDomain RBMotif
MW454557.1 22998: C>T SPIKE NT:1436 AA:479 -> P RBDomain RBMotif
MW454558.1 22998: C>T SPIKE NT:1436 AA:479 -> P RBDomain RBMotif
MW454562.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW454603.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW454683.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW454710.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW454757.1 23064: A>C SPIKE NT:1502 AA:501 -> N RBDomain RBMotif
MW449290.1 23036: T>C SPI

- Wie unterscheidet sich die Häufigkeit der Mutationen an verschiedenen Positionen?

In [161]:
! awk '$1 !~ /^#/{print $2}' result.txt | sort | uniq -c | sort -r | head -5

 999 23403:
 998 241:
 987 3037:
 984 14408:
 559 25563:


- Welche Nukleotidaustausche sind am häufigsten?

In [162]:
! awk '$1 !~ /^#/{print $3}' result.txt | sort | uniq -c | sort -r | head -5

6617 C>T
1665 G>T
1518 A>G
 998 G>A
 557 T>C


- Und welche betreffen am häufigsten die RBD?

In [163]:
! awk '$1 !~ /^#/ && $9 == "RBDomain"{print $2"_"$6"_"$9"_"$10}' result.txt | sort | uniq -c | sort -r | head -5

   6 23064:_AA:501_RBDomain_RBMotif
   4 23127:_AA:522_RBDomain_
   4 23122:_AA:521_RBDomain_
   4 22998:_AA:479_RBDomain_RBMotif
   3 23120:_AA:520_RBDomain_


Ein nächster interessanter Schritt wäre die Projektion dieser Daten auf die 3D-Struktur des Proteins. Das freiverfügbare Programm Jmol können Sie direkt runterladen:

In [ ]:
! wget 'https://sourceforge.net/projects/jmol/files/Jmol/Version%2014.31/Jmol%2014.31.29/Jmol-14.31.29-full.tar.gz'

In [156]:
! tar -xf Jmol-14.31.29-full.tar.gz jmol-14.31.29

- Sie können folgende Befehle in eine Textdatei _7DF4.script_schreiben und direkt mit Jmol ausführen als `./jmol-14.31.29/jmol.sh -s 7DF4.script &` ausführen

```
load =7DF4
spacefill off; wireframe off
cartoon
select :A; color lightgray # ACE2
select :C; color gray # Spike 2
select :D; color darkgray # Spike 3
select :B; color lightblue # Spike 1
select 319-541:B; color blue # RBDomain
select 437-508:B; color red # RBMotif
select 501:B,522:B,521:B,479:B,520:B # Mutations
spacefill 30
color yellow
```


| ![Virus DB](Images/spike.png)
|:---
| Das Spikeprotein (hellblau) mit der RBD in blau und dem RBMotif in rot. Violett ist der ACE2-Rezeptor dargestellt. Gelb und hervorgehoben die genetischen Varianten.
(PDB 7DF4; DOI 10.1126/sciadv.abe5575)

---

# Suche nach Mutationen mit BWA, SAM und BCF

Dieses Vorgehen ist für __fortgeschrittene__ Nutzer von Linux und erfolgert die __Installation__ mehrerer Software Tools. Allerdings ist diese Pipeline angelehnt an die Analyse von Sequenzdaten und die Suche nach SNPs und daher universeller einsetzbar. Sie brauchen aber eine Linuxumgebung mit Administratorrechten. Ich verwende dafür die im Mastermodul installierte __virtuelle Maschine mit Ubuntu__. 

Alternativ können Sie mit _Cygwin_ (https://www.cygwin.com) oder der _Ubuntu Emulator_ für Windows 10 (https://ubuntu.com/tutorials/ubuntu-on-windows#1-overview) verwendet werden.



- Als erstes Alignieren wir die SARS-CoV-2 Genome mit __BWA__ mit dem Referenzgenom. Dazu müssen wir einen Index erstellen, damit das Programm BWA auf die Daten zugreifen kann. Dabei werden einige Dateien erstellt.

In [145]:
! bwa index NC_045512.2.fasta

[bwa_index] Pack FASTA... 0.00 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 0.01 seconds elapse.
[bwa_index] Update BWT... 0.00 sec
[bwa_index] Pack forward-only FASTA... 0.00 sec
[bwa_index] Construct SA from BWT and Occ... 0.00 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index NC_045512.2.fasta
[main] Real time: 0.063 sec; CPU: 0.022 sec


- dann folgt das eigentliche Alignment

In [146]:
! bwa mem NC_045512.2.fasta cov2-len-29903.fasta > alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 336 sequences (10047408 bp)...
[M::process] read 336 sequences (10047408 bp)...
[M::mem_process_seqs] Processed 336 reads in 6.702 CPU sec, 6.688 real sec
[M::process] read 336 sequences (10047408 bp)...
[M::mem_process_seqs] Processed 336 reads in 7.182 CPU sec, 7.163 real sec
[M::process] read 143 sequences (4276129 bp)...
[M::mem_process_seqs] Processed 336 reads in 6.586 CPU sec, 6.570 real sec
[M::mem_process_seqs] Processed 143 reads in 2.335 CPU sec, 2.325 real sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem NC_045512.2.fasta cov2-len-29903.fasta
[main] Real time: 22.774 sec; CPU: 22.832 sec


- Jetzt muss die SAM-Datei mit __SamTools__ in eine binäre BAM-Datei umgewandelt werden. Die Option `-b` besagt, das die Ausgabe im BAM-Format sein soll.

In [147]:
! samtools view -b alignment.sam > alignment.sam.bam

- jetzt folgt ein Sortierschritt für den schnelleren Datenzugriff

In [148]:
! samtools sort alignment.sam.bam > alignment.sam.sorted.bam

- es folgen eine Indexierung des Referenzgenoms (`faidx`) und die Suche nach Sequenzunterschieden (`mpileup`) mit __SamTools__ und die Erstellung einer Datei im VCF-Format (_variant calling format_) mit __BcfTools__ (`view`)

In [149]:
! samtools faidx NC_045512.2.fasta

In [150]:
! bcftools mpileup -d 2000 -f NC_045512.2.fasta alignment.sam.sorted.bam > alignment.bcf

[mpileup] 1 samples in 1 input files
[mpileup] maximum number of reads per input file set to -d 2000


In [151]:
! bcftools call --variants-only --multiallelic-caller --ploidy 1 -o alignment.variants.vcf alignment.bcf

In [152]:
! awk '$1 !~ /^##/{OFS="\t"; print $2,$4,$5,$6}' alignment.variants.vcf

POS	REF	ALT	QUAL
241	C	T	228
3037	C	T	228
14408	C	T	228
23403	A	G	228


Das Ergbnis zeigt, dass bspw. an Position 241 im Referenzgenom ein C, in den meisten anderen Genomen aber ein T vorliegt. Die Qualität _QUAL_ liegt als sogenannter Phred-Score vor. Einer Fehlerwahrscheinlichkeit E von 10% entspricht ein Quality-Score Q von 10; von 1% -> 20; von 0.1% -> 30. Es gilt E=10^(-Q/10) und Q=-10xlogE.

Natürlich liefert uns diese Pipeline nur häufige SNPs, denn sie ist eigentlich für die Suche nach relevanten Mutationen (SNPs) in größeren Populationen gedacht.

Im Falle von Viren, können aber auch sehr seltene Varianten schnell Verbreitung finden. Diese finden wir zum Beispiel mit der oben dargestellten AWK-Variante.